In [15]:
import joblib
import os
import csv
import logging
import datetime
import pandas as pd
import numpy as np
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
import xgboost

/Users/luca/Documents/Proyectos personales/xtream-ai-assignment-developer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def load_data(file):
    """
    Load and preprocess data from a CSV file.
    
    Parameters:
        file (str): URL or local path to the CSV file containing data.
    
    Returns:
        pd.DataFrame: Preprocessed DataFrame with one-hot encoded categorical variables and filtered invalid entries.
    """
    diamonds = pd.read_csv(file)
    diamonds = diamonds[(diamonds.x * diamonds.y * diamonds.z != 0) & (diamonds.price > 0)]

    return diamonds

In [4]:
def preprocess_data(diamonds, model_type='linear'):
    if model_type == 'linear':
        return pd.get_dummies(diamonds, columns=['cut', 'color', 'clarity'], drop_first=True)
    elif model_type == 'tree':
        for col in ['cut', 'color', 'clarity']:
            diamonds[col] = pd.Categorical(diamonds[col])
        return pd.get_dummies(diamonds, columns=['cut', 'color', 'clarity'], drop_first=False)
    return diamonds

In [5]:
data_file = "https://raw.githubusercontent.com/xtreamsrl/xtream-ai-assignment-engineer/main/datasets/diamonds/diamonds.csv"
data = load_data(data_file) 

data_linear = preprocess_data(data.copy(), 'linear')
data_tree = preprocess_data(data.copy(), 'tree')

data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.10,Ideal,H,SI2,62.0,55.0,4733,6.61,6.65,4.11
1,1.29,Ideal,H,SI1,62.6,56.0,6424,6.96,6.93,4.35
2,1.20,Premium,I,SI1,61.1,58.0,5510,6.88,6.80,4.18
3,1.50,Ideal,F,SI1,60.9,56.0,8770,7.43,7.36,4.50
4,0.90,Very Good,F,VS2,61.7,57.0,4493,6.17,6.21,3.82


In [6]:
def split_data(data):
    """
    Split the data into training and testing sets.
    
    Parameters:
        data (pd.DataFrame): The DataFrame to split.
    
    Returns:
        tuple: Contains training and testing datasets (x_train, x_test, y_train, y_test).
    """
    x = data.drop(columns='price')
    y = data.price
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

In [12]:
x_train_linear, x_test_linear, y_train_linear, y_test_linear = split_data(data_linear)
x_train_tree, x_test_tree, y_train_tree, y_test_tree = split_data(data_tree)

In [8]:
def train_model(model, x_train, y_train):
    """
    Train a machine learning model.
    
    Parameters:
        model: The machine learning model to be trained.
        x_train (pd.DataFrame): Training data features.
        y_train (pd.Series): Training data labels.
    
    Returns:
        model: Trained model.
    """
    try:
        model.fit(x_train, y_train)
        logging.info(f"Training completed successfully for {type(model).__name__}")

        return model
    
    except Exception as e:
        logging.error(f"Error training model {type(model).__name__}: {str(e)}", exc_info=True)
        
        return None


In [9]:
def evaluate_model(model, x_test, y_test):
    """
    Evaluate a machine learning model using R2 and MAE.
    
    Parameters:
        model: The trained machine learning model to evaluate.
        x_test (pd.DataFrame): Testing data features.
        y_test (pd.Series): Testing data labels.
    
    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    try:
        predictions = model.predict(x_test)
        r2 = r2_score(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        logging.info(f"Evaluation metrics calculated successfully for {type(model).__name__}")

        return {"R2 Score": r2, "MAE": mae}
    
    except Exception as e:
        logging.error(f"Error evaluating model {type(model).__name__}: {str(e)}", exc_info=True)

        return None

In [10]:
def save_model(model, performance_metrics, model_name):
    """
    Save the model and its performance metrics with a timestamp in a txt file.
    """
    timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    model_directory = f"models/{model_name}/{timestamp}"

    os.makedirs(model_directory, exist_ok=True)
    joblib.dump(model, f"{model_directory}/model.joblib")

    with open(f"{model_directory}/metrics.txt", 'w') as f:
        f.write(str(performance_metrics))

In [11]:
def log_metrics(metrics, model_name, filename="model_performance.csv"):
    """
    Log model performance metrics to a CSV file.
    
    Parameters:
        metrics (dict): Dictionary containing the performance metrics.
        model_name (str): Name of the model.
        filename (str): Name of the CSV file to log the metrics.
    """
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['model_name', 'timestamp', 'R2_score', 'MAE']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        metrics_row = {
            'model_name': model_name,
            'timestamp': datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"),
            'R2_score': metrics['R2 Score'],
            'MAE': metrics['MAE']
        }

        writer.writerow(metrics_row)

In [19]:
def optimize_hyperparameters(x_train, y_train, n_trials=100):
    """
    Perform hyperparameter optimization using Optuna.

    Parameters:
        x_train, y_train (DataFrame, Series): Training data.
        n_trials (int): Number of optimization trials.

    Returns:
        dict: Best model parameters found.
    """
    def objective(trial):
        param = {
            'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.7]),
            'subsample': trial.suggest_categorical('subsample', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.2),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'random_state': 42,
            'objective': 'reg:squarederror'
        }
        model = xgboost.XGBRegressor(**param)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], verbose=False)
        preds = model.predict(x_train)
        mae = mean_absolute_error(y_train, preds)
        
        return mae

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    return study.best_params

In [17]:
models = {
    "LinearRegression": {"model": LinearRegression(), "type": "linear", "optimize": False},
    "Ridge": {"model": Ridge(), "type": "linear", "optimize": False},
    "Lasso": {"model": Lasso(), "type": "linear", "optimize": False},
    "ElasticNet": {"model": ElasticNet(), "type": "linear", "optimize": False},
    "LinearSVR": {"model": LinearSVR(), "type": "linear", "optimize": False},
    "SGDRegressor": {"model": SGDRegressor(), "type": "linear", "optimize": False},
    "XGBoost": {"model": xgboost.XGBRegressor(enable_categorical=True, random_state=42), "type": "tree", "optimize": True}
}

In [20]:
for model_name, config in models.items():
    model = config["model"]
    model_type = config["type"]
    should_optimize = config["optimize"]

    if model_type == "linear":
        x_train, x_test, y_train, y_test = x_train_linear, x_test_linear, y_train_linear, y_test_linear
    elif model_type == "tree":
        x_train, x_test, y_train, y_test = x_train_tree, x_test_tree, y_train_tree, y_test_tree

    logging.info(f"Starting training for {model_name}")
    
    if should_optimize:
        logging.info(f"Optimizing hyperparameters for {model_name}")
        best_params = optimize_hyperparameters(x_train, y_train)
        model.set_params(**best_params)

    trained_model = train_model(model, x_train, y_train)
    
    if trained_model:
        metrics = evaluate_model(trained_model, x_test, y_test)
        if metrics:
            save_model(trained_model, metrics, model_name)
            log_metrics(metrics, model_name)
            logging.info(f"{model_name} - R2 Score: {metrics['R2 Score']}, MAE: {metrics['MAE']}")
        else:
            logging.error(f"Failed to evaluate {model_name}", exc_info=True)
    else:
        logging.error(f"Failed to train {model_name}", exc_info=True)

2024-07-03 00:32:50,962 - INFO - Starting training for LinearRegression
2024-07-03 00:32:50,967 - INFO - Training completed successfully for LinearRegression
2024-07-03 00:32:50,969 - INFO - Evaluation metrics calculated successfully for LinearRegression
2024-07-03 00:32:50,970 - INFO - LinearRegression - R2 Score: 0.9090798105116309, MAE: 756.7317987073
2024-07-03 00:32:50,971 - INFO - Starting training for Ridge
2024-07-03 00:32:50,973 - INFO - Training completed successfully for Ridge
2024-07-03 00:32:50,975 - INFO - Evaluation metrics calculated successfully for Ridge
2024-07-03 00:32:50,976 - INFO - Ridge - R2 Score: 0.9089438362670546, MAE: 756.1527571184614
2024-07-03 00:32:50,977 - INFO - Starting training for Lasso
/Users/luca/Documents/Proyectos personales/xtream-ai-assignment-developer/venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check th

[0]	validation_0-rmse:3950.96094
[1]	validation_0-rmse:3907.46189
[2]	validation_0-rmse:3864.70486
[3]	validation_0-rmse:3821.89424
[4]	validation_0-rmse:3780.16111
[5]	validation_0-rmse:3739.18839
[6]	validation_0-rmse:3698.44690
[7]	validation_0-rmse:3658.36403
[8]	validation_0-rmse:3619.22698
[9]	validation_0-rmse:3580.37384
[10]	validation_0-rmse:3541.58333
[11]	validation_0-rmse:3503.55341
[12]	validation_0-rmse:3502.20792
[13]	validation_0-rmse:3465.33299
[14]	validation_0-rmse:3428.54196
[15]	validation_0-rmse:3392.68252
[16]	validation_0-rmse:3356.93305
[17]	validation_0-rmse:3322.20870
[18]	validation_0-rmse:3287.71422
[19]	validation_0-rmse:3253.67649


/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': tria

[20]	validation_0-rmse:3220.30423
[21]	validation_0-rmse:3187.23067
[22]	validation_0-rmse:3154.34168
[23]	validation_0-rmse:3122.33296
[24]	validation_0-rmse:3090.36224
[25]	validation_0-rmse:3059.15641
[26]	validation_0-rmse:3027.89281
[27]	validation_0-rmse:2997.65551
[28]	validation_0-rmse:2967.63060
[29]	validation_0-rmse:2938.51367
[30]	validation_0-rmse:2909.15209
[31]	validation_0-rmse:2880.65465
[32]	validation_0-rmse:2852.81567
[33]	validation_0-rmse:2825.23749
[34]	validation_0-rmse:2798.21017
[35]	validation_0-rmse:2771.02955
[36]	validation_0-rmse:2744.18390
[37]	validation_0-rmse:2718.12232
[38]	validation_0-rmse:2692.47210
[39]	validation_0-rmse:2667.12235
[40]	validation_0-rmse:2642.06164
[41]	validation_0-rmse:2617.44580
[42]	validation_0-rmse:2593.00504
[43]	validation_0-rmse:2568.91397
[44]	validation_0-rmse:2545.30370
[45]	validation_0-rmse:2522.05850
[46]	validation_0-rmse:2499.62697
[47]	validation_0-rmse:2476.53444
[48]	validation_0-rmse:2454.17091
[49]	validatio

[I 2024-07-03 00:32:52,567] Trial 0 finished with value: 370.20621388396415 and parameters: {'lambda': 0.5803521662481325, 'alpha': 0.007168256140879815, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.012673935765496606, 'n_estimators': 824, 'max_depth': 3, 'min_child_weight': 2}. Best is trial 0 with value: 370.20621388396415.


...

[19]	validation_0-rmse:592.55524
[20]	validation_0-rmse:589.68980
[21]	validation_0-rmse:533.77149
[22]	validation_0-rmse:477.86470
[23]	validation_0-rmse:429.80868
[24]	validation_0-rmse:390.67875
[25]	validation_0-rmse:351.49729
[26]	validation_0-rmse:321.09207
[27]	validation_0-rmse:288.27926
[28]	validation_0-rmse:259.19277
[29]	validation_0-rmse:244.26643
[30]	validation_0-rmse:229.65061
[31]	validation_0-rmse:209.13059
[32]	validation_0-rmse:188.68436
[33]	validation_0-rmse:176.21344
[34]	validation_0-rmse:159.47325
[35]	validation_0-rmse:146.93369
[36]	validation_0-rmse:135.10541
[37]	validation_0-rmse:125.63111
[38]	validation_0-rmse:123.43012
[39]	validation_0-rmse:118.59123
[40]	validation_0-rmse:112.91458
[41]	validation_0-rmse:105.96085
[42]	validation_0-rmse:101.19128
[43]	validation_0-rmse:93.67956
[44]	validation_0-rmse:93.33095
[45]	validation_0-rmse:89.45700
[46]	validation_0-rmse:88.70436
[47]	validation_0-rmse:84.34368
[48]	validation_0-rmse:78.50445
[49]	validation_

[I 2024-07-03 00:37:39,742] Trial 70 finished with value: 0.01699522472927368 and parameters: {'lambda': 1.8211699681096702e-06, 'alpha': 0.0006725084181338538, 'colsample_bytree': 0.3, 'subsample': 1.0, 'learning_rate': 0.1361822505427227, 'n_estimators': 427, 'max_depth': 20, 'min_child_weight': 2}. Best is trial 44 with value: 0.0010366341847281382.


[0]	validation_0-rmse:3326.70488
[1]	validation_0-rmse:2796.69392
[2]	validation_0-rmse:2720.17369
[3]	validation_0-rmse:2263.56044
[4]	validation_0-rmse:1921.14106
[5]	validation_0-rmse:1625.74935
[6]	validation_0-rmse:1524.27123
[7]	validation_0-rmse:1303.06679
[8]	validation_0-rmse:1100.69369
[9]	validation_0-rmse:1089.95075
[10]	validation_0-rmse:921.34805
[11]	validation_0-rmse:797.12063
[12]	validation_0-rmse:795.05150
[13]	validation_0-rmse:793.06509
[14]	validation_0-rmse:690.49329
[15]	validation_0-rmse:592.05698


/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
/var/folders/wm/c5dyqf8x7gggnw1381p0rf3m0000gn/T/ipykernel_6561/3303083810.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': tria

[16]	validation_0-rmse:530.93674
[17]	validation_0-rmse:478.45655
[18]	validation_0-rmse:433.22224
[19]	validation_0-rmse:385.78666
[20]	validation_0-rmse:383.33105
[21]	validation_0-rmse:343.47433
[22]	validation_0-rmse:304.22539
[23]	validation_0-rmse:274.64414
[24]	validation_0-rmse:249.19482
[25]	validation_0-rmse:220.86709
[26]	validation_0-rmse:199.52789
[27]	validation_0-rmse:179.01515
[28]	validation_0-rmse:160.16023
[29]	validation_0-rmse:152.00333
[30]	validation_0-rmse:142.87525
[31]	validation_0-rmse:128.03954
[32]	validation_0-rmse:115.34697
[33]	validation_0-rmse:107.89880
[34]	validation_0-rmse:97.82167
[35]	validation_0-rmse:89.16755
[36]	validation_0-rmse:80.22672
[37]	validation_0-rmse:74.37469
[38]	validation_0-rmse:72.72562
[39]	validation_0-rmse:70.11966
[40]	validation_0-rmse:67.29850
[41]	validation_0-rmse:63.62016
[42]	validation_0-rmse:61.19805
[43]	validation_0-rmse:56.85753
[44]	validation_0-rmse:56.70583
[45]	validation_0-rmse:53.76210
[46]	validation_0-rmse

---

In [50]:
help(load_data)

Help on function load_data in module __main__:

load_data(file)
    Load and preprocess data from a CSV file.

    Parameters:
        file (str): URL or local path to the CSV file containing data.

    Returns:
        pd.DataFrame: Preprocessed DataFrame with one-hot encoded categorical variables and filtered invalid entries.



In [52]:
help(split_data)

Help on function split_data in module __main__:

split_data(data)
    Split the data into training and testing sets.

    Parameters:
        data (pd.DataFrame): The DataFrame to split.

    Returns:
        tuple: Contains training and testing datasets (x_train, x_test, y_train, y_test).



In [54]:
help(train_model)

Help on function train_model in module __main__:

train_model(model, x_train, y_train)
    Train a machine learning model.

    Parameters:
        model (estimator): The machine learning model to be trained.
        x_train (pd.DataFrame): Training data features.
        y_train (pd.Series): Training data labels.

    Returns:
        model: Trained model.



In [56]:
help(evaluate_model)

Help on function evaluate_model in module __main__:

evaluate_model(model, x_test, y_test)
    Evaluate a machine learning model using R2 and MAE.

    Parameters:
        model: The trained machine learning model to evaluate.
        x_test (pd.DataFrame): Testing data features.
        y_test (pd.Series): Testing data labels.

    Returns:
        dict: Dictionary containing evaluation metrics.

